# 🛒 Retail y Consumo Masivo: demanda, surtido y pérdidas
KPIs: rotación, OSA, margen, shrink, fill rate, tiempo reposición.

## Casos de uso clave
- Forecast de demanda por tienda-SKU
- Optimización de inventarios y surtido
- Detección de pérdidas (shrinkage) y fraude
- Promociones personalizadas y canibalización

In [ ]:
# Esqueleto de tabla de hechos (ventas)
from dataclasses import dataclass
@dataclass
class Venta:
    fecha: str
    tienda_id: int
    sku: str
    unidades: int
    precio: float

def ingreso(v: Venta) -> float:
    return v.unidades * v.precio

Venta('2025-01-01', 10, 'SKU123', 2, 4.5), ingreso(Venta('2025-01-01', 10, 'SKU123', 2, 4.5))

## Arquitectura sugerida
- Ingesta POS/ERP y e-commerce (batch/stream).
- Modelo estrella: Hechos Ventas, Dimensión Producto/Tienda/Calendario.
- Serving: dashboards OSA, alertas shrink, API recomendaciones.

## 💼 Valor y palancas en Retail
- Menos quiebres de stock (OSA ↑) y mejor fill rate → ventas y satisfacción.
- Surtido óptimo por tienda-SKU → margen y rotación.
- Menos pérdidas (shrink/fraude) con alertas y controles.
- Promos efectivas sin canibalizar categorías clave.


In [ ]:
# Práctica: KPIs con ventas y productos (toy)
from pathlib import Path
import pandas as pd

# Intento de rutas relativas robustas
candidatos = [
    Path('../../datasets/raw/ventas.csv'),
    Path('../datasets/raw/ventas.csv'),
    Path('datasets/raw/ventas.csv'),
]
ventas_path = next((p for p in candidatos if p.exists()), candidatos[0])
prod_cands = [
    Path('../../datasets/raw/productos.csv'),
    Path('../datasets/raw/productos.csv'),
    Path('datasets/raw/productos.csv'),
]
productos_path = next((p for p in prod_cands if p.exists()), prod_cands[0])

try:
    ventas = pd.read_csv(ventas_path)
    productos = pd.read_csv(productos_path)
    # Ingreso bruto por mes y top SKUs
    ventas['fecha'] = pd.to_datetime(ventas['fecha']) if 'fecha' in ventas.columns else pd.to_datetime('today')
    ventas['ingreso'] = ventas.get('precio', 0) * ventas.get('cantidad', ventas.get('unidades', 1))
    mensual = ventas.groupby(pd.Grouper(key='fecha', freq='M'))['ingreso'].sum().reset_index()
    top_sku = ventas.groupby('producto_id')['ingreso'].sum().nlargest(5)
    print('Ingreso mensual (muestra):')
    print(mensual.head())
    print('\nTop 5 SKUs por ingreso:')
    print(top_sku)
except Exception as e:
    print('No se pudo cargar datasets de ejemplo:', e)
    print('Asegura la ruta relativa a datasets/raw/*.csv')


## 🏗️ De datos a decisiones
- Pipeline ELT: POS/ERP → limpieza → modelo estrella (Hechos Ventas, Dim Producto/Tienda/Calendario).
- Serving: dashboards OSA, alertas de shrink, microservicio de recomendaciones.
- Métricas confiables: definiciones comunes y control de calidad (tests de datos).
